In [29]:
#//----------------------------
#//LIBRARIES
    #Math
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('../')
from analysts_tools.growth import *
    #Procurement tools
from procurement_lib import send_slack_notification
from procurement_lib import redash
from analysts_tools.redash_methods import *

import random
import time

### Loading COSTUMERS INFO

In [30]:
#//---------------------------------------------------------------------------------------------------------------------------
#// Parametros

city_code = 'BOG'
    
#year-month-day
todays_date = '2022-10-15' 
#non_buyers_date = '2022-08-26' #1 months
customer_date_desde = '2022-09-05' #1 months
    #data for sku sales data
#sku_date_desde = '2022-06-26' #2 months

tasas = {'BOG': 3776,
         'BAQ': 3776,
         'MDE': 3776,
         'CMX': 19.65,
         'GDL': 19.65,
         'PBC': 19.65,
         'SPO': 4.75,
         'BHZ': 4.75,
         'CWB': 4.75,
         'VCP': 4.75}

t_cambio = tasas[city_code]

In [31]:
#//--------------------------------------
#//Ruta de data // KAM's costumers to delete
#//KAMs customers me lo manda Emiliano Lomeli (SE ACTUALIZAN DE UN SCRIPT MES A MES)
users_eliminar_kam = pd.read_csv("SD/import_data/segments/KAMs_OCT.csv")['costumer_id'].unique().tolist()
print(len(users_eliminar_kam))

2456


In [32]:
#Loading SELL'S DATA from a function
data_ventas = data_ventas_query(ciudad=city_code, tipo_cambio=t_cambio, desde=customer_date_desde, hasta = todays_date, tipo_negocio=1)

In [33]:
frida_products = get_fresh_query_result("https://internal-redash.federate.frubana.com/",48322,'SeoGHWmDUaaBi7VXje1s9zYNiMD1VHQ1K1DYOxiF',{},20)

200


### PARAMETERS // SOME FUNCTIONS - GET READY TO PROCESSING

In [34]:
active_skus = frida_products[frida_products.status_frida == 'on']["sku_id"].unique()
data_ventas = data_ventas[data_ventas.sku_id.isin(active_skus)]

data_ventas.shape

(493956, 19)

In [35]:
data_ventas = data_ventas.groupby(by=['region_code','customer_id','category','subcat','padre_sku_id']).agg({"name":pd.Series.unique, "gmv_usd":np.sum, "discount_applied":np.sum, "cant": np.sum, "order_id":pd.Series.nunique}).reset_index()

In [36]:
frida_products['sku_id'] = frida_products['sku_id'].astype(str)

In [37]:
frida_products_clean = frida_products[(frida_products.status_frida == 'on') & (frida_products["ciudad::multiFilter"] == city_code)].groupby(by= ['id_tarjeta']).apply(lambda x: '#'.join(x["sku_id"].unique())).reset_index()
frida_products_clean = frida_products_clean.rename(columns={0:'skus'})

In [38]:
data_ventas_df = pd.merge(data_ventas,frida_products_clean, left_on='padre_sku_id', right_on='id_tarjeta')#.drop(columns = ['id_tarjeta'])

In [39]:
def skus_offer(skus):
    
    skus_aux = [frida_products_clean.loc[frida_products_clean['skus'].str.contains(sk, case=False)]['skus'].to_list()[0] for sku in skus for sk in sku]
    
    a=[]*len(skus)
    contador=0
    contador2=0
    for i,sku in enumerate(skus):
        contador+= len(sku)
        a.append(skus_aux[contador2:contador])
        contador2+= len(sku)
        
    return a

In [40]:
def list_users(data_ventas, rangos):
    """This function select the customer_ids of the clients who are between the range of gmv_usd"""
    
    total_users = data_ventas["customer_id"].unique().tolist()
    print(f'Total users: {len(total_users)}')
    
    inferior, superior = np.percentile(data_ventas['gmv_usd'],rangos) #rango intercuartil entre los skus a tomar
    
    users = data_ventas[(data_ventas.gmv_usd>=inferior) & (data_ventas.gmv_usd<=superior)]['customer_id'].unique()
    print(f'Total users: {len(users)}')
    
    return users

In [41]:
data_ventas_df.subcat.unique()

array(['Arroz', 'Granos', 'Pasta', 'Salsas',
       'Desinfectantes & Sanitizantes', 'Detergente, Jabón & Lavalozas',
       'Cervezas & Licores', 'Pollo', 'Res', 'Alimentos Congelados',
       'Contenedores', 'Frutas', 'Tubérculos', 'Verduras', 'Cremas',
       'Aceites & Grasas', 'Azúcar & Endulzantes',
       'Café, Chocolate e Infusiones', 'Vasos', 'Harinas & Mezclas',
       'Implementos de Aseo', 'Papel & Toallas', 'Bolsas', 'Gaseosas',
       'Cerdo', 'Cubiertos & Pitillos', 'Frutas Jugo',
       'Platos & Bandejas', 'Aguas', 'Pulpas de Fruta',
       'Derivados Lácteos', 'Huevos', 'Enlatados', 'Leches',
       'Sal & Sazonadores', 'Mantequillas', 'Jugos',
       'Pescados & Mariscos Congelado', 'Complementos',
       'Panadería & Repostería', 'Embutidos', 'Dotación', 'Quesos', 'Tés',
       'Empaque & Envoltura', 'Especias', 'Mesa', 'Utensilios de cocina',
       'Isotónicos', 'Cristalería y Vajilla', 'Cuchillos y Cubertería',
       'Ollas y Sartenes', 'Electro'], dtype=object

# Customize Offers

### Load the information in the next block

In [42]:
"""Here we should complete our data for each SH, important if we want to submit a subcategory SH offer, in SKUS we should put the skus into a list as this [[1,2,3]]"""
offers_name = ['Harina','Esmeralda','Moppy']
skus = [['254112'],['401541'],['304652','409396','304650','304644','304642','409394','304648','304646','409392']] #it should be the sku in string
subcat = [['Harinas & Mezclas'],['Leches'],['Desinfectantes & Sanitizantes']]
discount = [6,5,11]
max_uses = [5,40,40]
max_orders = [2,2,2]
#////////////
rango_IQ = [[0,90],[0,90],[0,90]]
users = []

In [43]:
for j, sku in enumerate(skus):
    range = rango_IQ[j]
    print(range)
    
    if len(sku) == 0:
        x = []
        aux_data = data_ventas_df.loc[(~data_ventas_df['skus'].str.contains(sku[0], case=False)) & (data_ventas_df['subcat'].str.contains(subcat[j][0], case=False))]
        x.append(list_users(aux_data,range))
    else:
        x = []
        y = []
        for i in sku:
            aux_data = data_ventas_df.loc[(~data_ventas_df['skus'].str.contains(i, case=False)) & (data_ventas_df['subcat'].str.contains(subcat[j][0], case=False))]
            y.append(aux_data)
        aux_data_all = pd.concat(y)
        aux_data_all = aux_data_all.groupby(by=['region_code','customer_id','category','subcat']).agg({"gmv_usd":np.sum, "discount_applied":np.sum, "cant": np.sum, "order_id":pd.Series.nunique}).reset_index()
        x.append(list_users(aux_data_all,range))
    users.append(x)

[0, 90]
Total users: 4007
Total users: 3606
[0, 90]
Total users: 1553
Total users: 1397
[0, 90]
Total users: 3816
Total users: 3434


# Generate CSV for Offers

In [44]:
global_segment_list = [] #//for procesing
r= {}

for i,offer in enumerate(offers_name):
    id_list = users[i][0].tolist()
    test = random.sample(list(id_list), int(len(id_list)*0.85))
    control = [value for value in id_list if value not in test]
    d = {'name':"SH_"+todays_date+"_"+city_code+"_"+offer, 'customersIds': list(test)}

    r['CONTROL_'+"SH_"+todays_date+"_"+city_code+"_"+offer] = control
    r['TEST_'+"SH_"+todays_date+"_"+city_code+"_"+offer] = test
    
    global_segment_list.append(d) 

df_test_control = pd.DataFrame.from_dict(r, orient='index').T

In [48]:
csv_file = {} 
csv_file['offer_name'] = ["SH_"+todays_date+"_"+city_code+"_"+x for x in offers_name]
csv_file['offer_description'] = None
csv_file['discount'] = discount
csv_file['automatically_consider_offer'] = "true"
csv_file['start_date'] = '15-10-2022'
csv_file['end_date'] = '22-10-2022'
csv_file['max_uses_per_order'] = max_uses
csv_file['max_uses_per_customer'] = max_orders
csv_file['customer_segment_id'] = [88052,88053,88054] #[None]
csv_file['sku_id'] = ["#".join(s) for s in skus_offer(skus)]

global_offer_csv = pd.DataFrame(csv_file) #, index=[0])

In [49]:
global_offer_csv

,offer_name,offer_description,discount,automatically_consider_offer,start_date,end_date,max_uses_per_order,max_uses_per_customer,customer_segment_id,sku_id
0,SH_2022-10-15_BOG_Harina,None,6,true,15-10-2022,22-10-2022,5,2,88052,254112#254113
1,SH_2022-10-15_BOG_Esmeralda,None,5,true,15-10-2022,22-10-2022,40,2,88053,324143#401541
2,SH_2022-10-15_BOG_Moppy,None,11,true,15-10-2022,22-10-2022,40,2,88054,304652#409396#304650#304644#304642#409394#3046...


In [50]:
#//--------------------------------------------
#//Exporting single file data
with open(f'SD/export_data/Below_Media/{city_code}/{todays_date}_penetration_{city_code}_All_SH.txt', 'w') as outfile:
        json.dump(global_segment_list, outfile)

In [51]:
#//--------------------------------------------
#//Exporting single file data
global_offer_csv.to_csv(f'SD/export_data/Below_Media/{city_code}/{todays_date}_OFFER_csv_{city_code}_All_SH_penetration.csv', index = False)
df_test_control.to_csv(f'SD/export_data/Below_Media/{city_code}/{todays_date}_CLIENTS_csv_{city_code}_All_SH_penetration.csv', index = False)